In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import matplotlib.pyplot as plt

try:
    import pyfftw as fftw
except:
    pass


import scipy as sp
import numpy as np

import llops as yp
import llops.operators as ops

yp.setDefaultDatatype('complex32')

# Numpy-Based Operators

## Create Object

In [3]:
sz = (2000, 2000)
x = yp.rand(sz)

## CPU/Numpy FFT Function Test

In [4]:
fftw.interfaces.cache.enable()

def numpy_fft(x):
    return np.fft.fft2(x) / sz[0]

def scipy_fft(x):
    return sp.fftpack.fft2(x) / sz[0]

def fftw_fft(x):
    return fftw.interfaces.numpy_fft.fft2(x) / sz[0]

for f in [numpy_fft, scipy_fft, fftw_fft]:
    print(f)
    %timeit f(x)
    %timeit f(f(f(x)))
    %timeit f(x) + f(x) + f(x)

<function numpy_fft at 0x116f95a60>
25.1 ms ± 476 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
63.4 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
73 ms ± 549 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
<function scipy_fft at 0x116f95c80>
142 ms ± 612 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
396 ms ± 9.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
435 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
<function fftw_fft at 0x11702a7b8>
73.8 ms ± 748 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
279 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
232 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## CPU/Numpy FFT Operator Test

In [9]:
F_np = ops.FourierTransform(sz, backend='numpy', fft_backend='numpy', center=False, normalize=False)
F_sp = ops.FourierTransform(sz, backend='numpy', fft_backend='scipy', center=False, normalize=False)
F_fftw = ops.FourierTransform(sz, backend='numpy', fft_backend='fftw', center=False, normalize=False)

F_prod = (F * F * F)
F_sum = F + F + F
for F in [F_np, F_sp, F_fftw]:
    print(F)
    %timeit F * x
    %timeit F_prod * x
    %timeit F_sum * x
    

<F: (2000, 2000)x(2000, 2000) FourierTransform NUMPY operator with dtype=complex32>
29 ms ± 510 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
291 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
337 ms ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
<F: (2000, 2000)x(2000, 2000) FourierTransform NUMPY operator with dtype=complex32>
172 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
284 ms ± 6.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
323 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
<F: (2000, 2000)x(2000, 2000) FourierTransform NUMPY operator with dtype=complex32>
97 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
279 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
316 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test Bottlenecks

In [ ]:
F = ops.FourierTransform(sz, backend='numpy', fft_backend='numpy', center=False, normalize=False)
Fc = ops.FourierTransform(sz, backend='numpy', fft_backend='numpy', center=True, normalize=False)
_x = yp.rand(sz)
y = yp.zeros_like(_x)

%timeit np.fft.fft2(_x) / 1000
# # %timeit yp.Ft(_x)
# %timeit F.fft_fun(_x)
# %timeit Fc.fft_fun(_x)
%timeit F._forward(_x, y)
%timeit F.forward(_x, y)
%timeit F * _x

In [ ]:
sz_0 = (100,100)
sz_1 = (50,50)

# Create fake object
x = yp.rand(sz_0)
# x[sz_1[0] // 4: 3* sz_1[0] // 4, sz_1[1] // 4: 3* sz_1[1] // 4] = yp.rand((sz_1[0] // 2, sz_1[1] // 2))
x /= yp.mean(yp.abs(x))

# Generate FFT operators
F_list_n = []
F_list_un = []
for fft_backend in yp.valid_fft_backends:
    F_list_n.append(ops.FourierTransform(sz_0, backend='numpy', fft_backend=fft_backend, center=True, normalize=True))
    F_list_un.append(ops.FourierTransform(sz_0, backend='numpy', fft_backend=fft_backend, center=True, normalize=False))

# Check inversion
assert all([yp.scalar(yp.sum(yp.abs(F.H * F * x - x))) < 1e-6 for F in F_list_un])
assert all([yp.scalar(yp.sum(yp.abs(F.H * F * x - x))) < 1e-6 for F in F_list_n])

# Check energy in Fourier domain
x_energy = yp.sum(yp.abs(x) ** 2)
energies_unnormalized = [yp.sum(yp.abs(F * x) ** 2) for F in F_list_n]
energies_normalized = [yp.sum(yp.abs(F * x) ** 2) for F in F_list_un]

print(x_energy)
print(energies_normalized)
print(energies_unnormalized)

assert all([abs(energy_normalized / yp.size(x) - x_energy) < 1e-1 for energy_normalized in energies_normalized])
assert all([abs(energy_unnormalized - x_energy) < 1e-4 * x_energy for energy_unnormalized in energies_unnormalized])

## Convolution Test

In [ ]:
# Create blur kernel
h = yp.zeros(sz_0)
h[sz_1[0] // 2:sz_1[0] // 2 + 1, sz_1[1] // 4: 3* sz_1[1] // 4,] = 1

# Create fake object
x = yp.zeros(h.shape)
x[sz_1[0] // 4: 3* sz_1[0] // 4, sz_1[1] // 4: 3* sz_1[1] // 4] = yp.rand((sz_1[0] // 2, sz_1[1] // 2))

# Generate forward operators
A_list = [F.H * ops.Diagonalize(F * h) * F for F in F_list_un]

# Generate measurements
y_list = [A * x for A in A_list]
y_sum = [yp.scalar(yp.sum(yp.abs(A * x))) for A in A_list]

# Check values
assert all([delta < 1e-3 for delta in yp.sum(yp.abs(x)) * yp.sum(yp.abs(h)) - y_sum])

plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(np.abs(x))
plt.colorbar()
plt.subplot(122)
plt.imshow(np.abs(y_list[0]))
plt.colorbar()

# Arrayfire-Based Operators

In [ ]:
sz = (2000, 2000)

## GPU/Arrayfire FFT Functon test

In [ ]:
if 'opencl' in af.get_available_backends():
    print('OpenCL:')
    af.set_backend('opencl')
    I2 = yp.rand(sz, backend='arrayfire')
    %timeit af.signal.fft(I2)

if 'cuda' in af.get_available_backends():
    print('CUDA:')
    af.set_backend('cuda')
    I3 = yp.rand(sz, backend='arrayfire')
    %timeit af.signal.fft(I3)

In [ ]:
%timeit af.signal.fft2_inplace(I2)

In [ ]:
x_af = yp.rand(sz, backend='arrayfire')
y_af = yp.zeros(sz, backend='arrayfire')
# %timeit af.signal.fft(x_af)
# %timeit af.signal.fft(x_af, scale=10)


F = ops.FourierTransform(sz, backend='arrayfire', center=False, normalize=False)
%timeit F._forward(x_af, y_af)
%timeit af.signal.fft2_inplace(I2)
# F.f
# F_ocl.forward(x_af)
# x_af = yp.rand(sz, backend='arrayfire')
# F_ocl.forward(I2)

## GPU/Arrayfire Operator Test

In [ ]:
# af.set_backend('cpu:')
# I1 = yp.rand(sz, backend='arrayfire')
# F_cpu = ops.FourierTransform(sz, backend='arrayfire', center=False, normalize=False, pad=False)
# print(af.get_backend_id(I1))
# %timeit F_cpu * I1

af.set_backend('opencl')
I2 = yp.rand(sz, backend='arrayfire')
F_ocl = ops.FourierTransform(sz, backend='arrayfire', center=False, normalize=False)
print(af.get_backend_id(I2))
%timeit F_ocl * I2

# af.set_backend('cuda')
# I3 = yp.rand(sz, backend='arrayfire')
# F_cuda = ops.FourierTransform(sz, backend='arrayfire', center=False, normalize=False, pad=False)
# print(af.get_backend_id(I3))
# %timeit F_cuda * I3

## GPU/Arrayfire Normalization Test

In [ ]:
backend = 'numpy'
sz_0 = (100,100)
sz_1 = (50,50)

# Create fake object
x = yp.zeros(sz_0, backend=backend)
x[sz_1[0] // 4: 3* sz_1[0] // 4, sz_1[1] // 4: 3* sz_1[1] // 4] = yp.rand((sz_1[0] // 2, sz_1[1] // 2), backend=backend)
x /= yp.mean(yp.abs(x))

# Generate FFT operators
F_list_n = []
F_list_un = []

for fft_backend in yp.valid_fft_backends:
    F_list_n.append(ops.FourierTransform(sz_0, backend=backend, fft_backend=fft_backend, center=True, normalize=True, pad=False))
    F_list_un.append(ops.FourierTransform(sz_0, backend=backend, fft_backend=fft_backend, center=True, normalize=False, pad=False))

# Check energy in Fourier domain
x_energy = yp.sum(yp.abs(x) ** 2)
energies_unnormalized = [yp.sum(yp.abs(F * x) ** 2) / yp.size(x) for F in F_list_n]
energies_normalized = [yp.sum(yp.abs(F * x) ** 2) / yp.size(x) for F in F_list_un]

print(x_energy)
print(energies_unnormalized)
print(energies_normalized)

assert all([abs(energy_normalized - x_energy) < 1e-5 * x_energy for energy_normalized in energies_normalized])
assert all([abs(energy_unnormalized * yp.size(x) - x_energy) < 1e-4 * x_energy for energy_unnormalized in energies_unnormalized])

# Check inversion
assert all([yp.scalar(yp.sum(yp.abs(F.H * F * x - x))) < 1e-6  * x_energy for F in F_list_un])
assert all([yp.scalar(yp.sum(yp.abs(F.H * F * x - x))) < 1e-6  * x_energy for F in F_list_n])



## Numpy / Arrayfire Transform Similarity

In [ ]:
normalize = False
sz = yp.shape(x)
x = yp.changeBackend(x, 'arrayfire')
F_af = ops.FourierTransform(sz, center=True, backend='arrayfire', normalize=normalize)
F_np = ops.FourierTransform(sz, center=True, backend='numpy', normalize=normalize)

plt.figure()
plt.subplot(221)
plt.imshow(yp.abs(yp.changeBackend(F_af * x, 'numpy')))
plt.title('Foward, af')
plt.colorbar()

plt.subplot(222)
plt.imshow(yp.abs(F_np * yp.changeBackend(x, 'numpy')))
plt.title('Foward, numpy')
plt.colorbar()

plt.subplot(223)
plt.imshow(yp.abs(yp.changeBackend(F_af.H * x, 'numpy')))
plt.title('Inverse, af')
plt.colorbar()

plt.subplot(224)
plt.imshow(yp.abs(F_np.H * yp.changeBackend(x, 'numpy')))
plt.title('Inverse, numpy')
plt.colorbar()

## Convolution Test

In [ ]:
backend = 'arrayfire'

h = yp.zeros(sz_0, backend=backend)
h[sz_0[0] // 4 * 3, sz_0[1] // 4 * 3] = 1

# Create blur kernel
h = yp.changeBackend(h, backend)
h /= yp.sum(yp.abs(h))

# Generate forward operators
F = ops.FourierTransform(sz_0, backend=backend, center=True, normalize=False, pad=False)
A = F.H * ops.Diagonalize(h, inside_operator=F) * F

# Generate measurements
y = A * yp.changeBackend(x, backend)
y_sum = yp.scalar(yp.sum(yp.abs(y)))

# Check values
assert yp.scalar(yp.sum(yp.abs(x)) * yp.sum(yp.abs(h))) - y_sum

plt.figure(figsize=(11,4))
plt.subplot(131)
plt.imshow(np.abs(x))
plt.colorbar()
plt.subplot(132)
plt.imshow(np.abs(h))
plt.colorbar()
plt.subplot(133)
plt.imshow(np.abs(y))
plt.colorbar()

In [ ]:
h = yp.zeros(sz_0)
h[sz_1[0] // 2:sz_1[0] // 2 + 1, sz_1[1] // 4: 3* sz_1[1] // 4,] = 1

# Create blur kernel
h = yp.changeBackend(h, 'arrayfire')
h /= yp.sum(yp.abs(h))

# Generate forward operators
F = ops.FourierTransform(sz_0, backend='arrayfire', center=False, normalize=False, pad=False)
A = F.H * ops.Diagonalize(h, inside_operator=F) * F

# Generate measurements
y = A * x
y_sum = yp.scalar(yp.sum(yp.abs(y)))

# Check values
assert yp.scalar(yp.sum(yp.abs(x)) * yp.sum(yp.abs(h))) - y_sum

plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(np.abs(x))
plt.colorbar()
plt.subplot(122)
plt.imshow(np.abs(y))
plt.colorbar()